In [21]:
import re
import random
import time
from statistics import mode
import json 

from PIL import Image
import numpy as np
import pandas
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [22]:
import matplotlib.pyplot as plt
from einops.layers.torch import Rearrange
from einops import rearrange
from einops import repeat

import random

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


In [24]:
def process_text_adv(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10', 
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't", "whats": "what's",
        "thats": "that's", "whos": "who's", "wheres": "where's", "whens": "when's",
        "please":"", "could you": "can you", "could i": "can i", "could we": "can we",
    }

    contractions_2 = {
        "theatre" : "theater", "colour" : "color", "centre" : "center", "favourite" : "favorite",
        "travelling" : "traveling", "counselling" : "counseling", "metre" : "meter",
        "cancelled" : "canceled", "labour" : "labor", "organisation" : "organization",
        "calibre" : "caliber", "cheque" : "check", "manoeuvre" : "maneuver",
        "neighbour" : "neighbor", "grey" : "gray", "dialogue" : "dialog",
    }

    contractions_3 = {
        "what is": "what's", "who is": "who's", "where is": "where's", "when is": "when's",
        "how is": "how's", "it is": "it's", "he is": "he's", "she is": "she's",
        "that is": "that's", "there is": "there's", "here is": "here's",
        "i am": "i'm", "you are": "you're", "we are": "we're", "they are": "they're",
        "i have": "i've", "you have": "you've", "we have": "we've", "they have": "they've",
        "i will": "i'll", "you will": "you'll",
    }
    
    # contractions_3 = {
    #     "what's" : "what is", "who's" : "who is", "where's" : "where is", "when's" : "when is",
    #     "how's" : "how is", "it's" : "it is", "he's" : "he is", "she's" : "she is",
    #     "that's" : "that is", "there's" : "there is", "here's" : "here is",
    #     "i'm" : "i am", "you're" : "you are", "we're" : "we are", "they're" : "they are",
    #     "i've" : "i have", "you've" : "you have", "we've" : "we have", "they've" : "they have",
    #     "i'll" : "i will", "you'll" : "you will",
    # }
 
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_2.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_3.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [25]:
def process_text_adv(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10', 
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't", "whats": "what's",
        "thats": "that's", "whos": "who's", "wheres": "where's", "whens": "when's",
        "please":"", "could you": "can you", "could i": "can i", "could we": "can we",
    }

    contractions_2 = {
        "theatre" : "theater", "colour" : "color", "centre" : "center", "favourite" : "favorite",
        "travelling" : "traveling", "counselling" : "counseling", "metre" : "meter",
        "cancelled" : "canceled", "labour" : "labor", "organisation" : "organization",
        "calibre" : "caliber", "cheque" : "check", "manoeuvre" : "maneuver",
        "neighbour" : "neighbor", "grey" : "gray", "dialogue" : "dialog",
    }

    contractions_3 = {
        "what is": "what's", "who is": "who's", "where is": "where's", "when is": "when's",
        "how is": "how's", "it is": "it's", "he is": "he's", "she is": "she's",
        "that is": "that's", "there is": "there's", "here is": "here's",
        "i am": "i'm", "you are": "you're", "we are": "we're", "they are": "they're",
        "i have": "i've", "you have": "you've", "we have": "we've", "they have": "they've",
        "i will": "i'll", "you will": "you'll",
    }
    # contractions_3 = {
    #     "what's" : "what is", "who's" : "who is", "where's" : "where is", "when's" : "when is",
    #     "how's" : "how is", "it's" : "it is", "he's" : "he is", "she's" : "she is",
    #     "that's" : "that is", "there's" : "there is", "here's" : "here is",
    #     "i'm" : "i am", "you're" : "you are", "we're" : "we are", "they're" : "they are",
    #     "i've" : "i have", "you've" : "you have", "we've" : "we have", "they've" : "they have",
    #     "i'll" : "i will", "you'll" : "you will",
    # }
 
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_2.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_3.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [26]:
def process_text(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't"
    }
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [27]:
def original_data(df_path, image_dir="", transform=None, answer=False):

    df = pandas.read_json(df_path)

    # question / answerの辞書を作成
    question2idx = {}
    answer2idx = {}
    questionInitialWords = dict()
    c1 = c2 = c3 = c4 = c5 = c6 = c7 = 0

    first_three_words = []
    first_four_words = []

    question_list = []

    # 質問文に含まれる単語を辞書に追加
    for question in df["question"]:
        question = process_text_adv(question)
        question_list.append(question)
        words = question.split(" ")

        if len(words) > 2:
            first_three_words.append(words[0] + " " + words[1] + " " + words[2])
            if len(words) > 3:
                first_four_words.append(words[0] + " " + words[1] + " " + words[2] + " " + words[3])
            
        for idex, word in enumerate(words):
            if idex == 0:
                if word not in questionInitialWords:
                    questionInitialWords[word] = 1
                else:
                    questionInitialWords[word] += 1            
            if word not in question2idx:
                question2idx[word] = len(question2idx)
    idx2question = {v: k for k, v in question2idx.items()}  # 逆変換用の辞書(question)

    if answer:
        # 回答に含まれる単語を辞書に追加
        for answers in df["answers"]:
            for answer in answers:
                word = answer["answer"]
                word = process_text(word)
                if word not in answer2idx:
                    answer2idx[word] = len(answer2idx)
        idx2answer = {v: k for k, v in answer2idx.items()}  # 逆変換用の辞書(answer)

    return first_three_words, first_four_words, question_list

In [28]:
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [29]:
# "answers":{"0":[{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"flat iron beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"steak"},{"answer_confidence":"yes","answer":"flat iron beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"}],"1":[{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"candle"},{"answer_confidence":"no","answer":"unanswerable"},{"answer_confidence":"maybe","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"no","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"}],

In [30]:
import json

# COCO_val2014_000000393075

answer_type = dict()
extra_answer_output = dict()
val_extra_answer_output = dict()

print("test")
# データセットの読み込み
with open('./extra_data/train_question.json', 'r') as f:
    question_data = json.load(f)

with open('./extra_data/train_answer.json', 'r') as f:
    answer_data = json.load(f)

extra_questions_data = question_data["questions"]
extra_answer_data = answer_data["annotations"]

test


In [31]:
three_words_1, four_words_1, question_list_1 = original_data('./data/train.json')
three_words_2, four_words_2, question_list_2 = original_data('./data/valid.json')

In [32]:
selected_question_list = []

question_map_dict = dict()
image_dict = dict()
question_dict = dict()
final_dict = dict()


val_selected_question_list = []

val_question_map_dict = dict()
val_image_dict = dict()
val_question_dict = dict()
val_final_dict = dict()



question_counter = 0
lee_value_counter = 0
val_counter = 0

In [33]:
print("test")
print(len(extra_questions_data))
print(extra_questions_data[0])

start_number = 19873
val_start_number = 0


counter = counter_four = counter_only_three = 0



dice = random.randint(1,6)

for data in tqdm(extra_questions_data):
    
    question = data["question"]
    parsed_question = process_text_adv(question)

    original_image_id = data["image_id"]
    question_id = data["question_id"]

    image_name = "COCO_val2014_" + str(original_image_id).zfill(12) + ".jpg"

    new_first_three_words = ""
    new_first_four_words = ""

    question_split = parsed_question.split(" ")

    iscontinue = True

    is_less_value = True

    first_three_words = ""
    
    if len(question_split) >2:
        first_three_words = question_split[0] + " " + question_split[1] + " " + question_split[2]
    
    if len(question_split) >3:
        first_four_words = question_split[0] + " " + question_split[1] + " " + question_split[2] + " " + question_split[3]

    if not (parsed_question in question_list_2):

        if first_three_words in three_words_2:
            is_less_value = False
            lee_value_counter += 1
        else:
            continue
    
    if not is_less_value:
        dice = random.randint(1,34)
        if dice < 23:
            continue

    if not is_less_value:
        dice = random.randint(1,6)
        if dice < 3:
            val_counter += 1
            val_selected_question_list.append(question_id)

            val_question_map_dict[question_id] = val_start_number
            val_image_dict[val_start_number] = image_name
            val_re_question_id = val_start_number

            val_question_dict[val_re_question_id] = question

            val_start_number += 1
            continue

    question_counter += 1

    selected_question_list.append(question_id)
    
    question_map_dict[question_id] = start_number
    image_dict[start_number] = image_name
    re_question_id = start_number
    
    question_dict[re_question_id] = question

    start_number += 1


final_dict["image"] = image_dict
final_dict["question"] = question_dict

val_final_dict["image"] = val_image_dict
val_final_dict["question"] = val_question_dict

print(extra_answer_data[0])

print(f"question_counter: {question_counter}")
print(f"less_value_counter: {lee_value_counter}")
print(f"val_counter: {val_counter}")

test
214354
{'image_id': 262148, 'question': 'Where is he looking?', 'question_id': 262148000}


100%|██████████| 214354/214354 [00:19<00:00, 10894.42it/s]

{'question_type': 'none of the above', 'multiple_choice_answer': 'down', 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}], 'image_id': 262148, 'answer_type': 'other', 'question_id': 262148000}
question_counter: 14174
less_value_counter: 45605
val_counter: 5383


In [34]:
# with open('./extra_data/test_question.json', 'w') as f:
#     json.dump(question_dict, f)

In [35]:
# with open('./extra_data/test_image.json', 'w') as f:
#     json.dump(image_dict, f)

In [36]:
min_image_id = 999999999999999999

answer_counter = 0
val_answer_counter = 0

for data in tqdm(extra_answer_data):
    question_id = data["question_id"]

    if question_id not in selected_question_list and question_id not in val_selected_question_list:
        continue

    if question_id in selected_question_list:

        re_question_id = question_map_dict[question_id]

        image_id = data["image_id"]
        if image_id < min_image_id:
            min_image_id = image_id
        answer_list = data['answers']
        replace_answer_list = []
        for answer in answer_list:
            answer_data = dict()
            answer_data["answer_confidence"] = answer["answer_confidence"]
            answer_data["answer"] = answer["answer"]
            answer = answer_data
            replace_answer_list.append(answer_data)
        extra_answer_output[re_question_id] = replace_answer_list
        answer_counter += 1
    
    if question_id in val_selected_question_list:
        re_question_id = val_question_map_dict[question_id]

        image_id = data["image_id"]
        if image_id < min_image_id:
            min_image_id = image_id
        answer_list = data['answers']
        replace_answer_list = []
        for answer in answer_list:
            answer_data = dict()
            answer_data["answer_confidence"] = answer["answer_confidence"]
            answer_data["answer"] = answer["answer"]
            answer = answer_data
            replace_answer_list.append(answer_data)
        val_extra_answer_output[re_question_id] = replace_answer_list
        val_answer_counter += 1

final_dict["answers"] = extra_answer_output
val_final_dict["answers"] = val_extra_answer_output

print(f"min_image_id: {min_image_id}")
print(f"answer_counter: {answer_counter}")
print(f"val_answer_counter: {val_answer_counter}")

100%|██████████| 214354/214354 [00:35<00:00, 5954.84it/s]

min_image_id: 42
answer_counter: 14174
val_answer_counter: 5383


In [37]:
with open('./extra_data/test.json', 'w') as f:
    json.dump(extra_answer_output, f, indent=2)

with open('./extra_data/extra_train_data_v4.json', 'w') as f:
    json.dump(final_dict, f)

with open('./extra_data/extra_valid_data_v4.json', 'w') as f:
    json.dump(val_final_dict, f)

In [38]:
import pandas

In [39]:
from tqdm import tqdm

In [40]:
train_dataset = VQADataset(df_path="./data/train.json", image_dir="./data/train", transform=transform_test)

NameError: name 'VQADataset' is not defined